<a href="https://colab.research.google.com/github/hr1588/dacon/blob/main/%EC%98%A4%EB%8A%98%EC%9D%98_%ED%8C%8C%EC%9D%B4%EC%8D%AC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [서울시 따릉이 자전거 이용 예측 AI 모델 ]

- 어제 의사결정나무를 활용해 만든 모델의 하이퍼 파라미터를 조정해 보았습니다.

In [1]:
# 필요한 패키지 로드

import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [2]:
# 데이터 다운로드

!wget 'https://bit.ly/3gLj0Q6'

import zipfile
with zipfile.ZipFile('3gLj0Q6', 'r') as existing_zip:
    existing_zip.extractall('data')

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')


--2022-07-27 11:11:23--  https://bit.ly/3gLj0Q6
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1or_QN1ksv81DNog6Tu_kWcZ5jJWf5W9E [following]
--2022-07-27 11:11:23--  https://drive.google.com/uc?export=download&id=1or_QN1ksv81DNog6Tu_kWcZ5jJWf5W9E
Resolving drive.google.com (drive.google.com)... 172.253.115.113, 172.253.115.139, 172.253.115.100, ...
Connecting to drive.google.com (drive.google.com)|172.253.115.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ed8a5tt7naegr6m355026hfhfencaknq/1658920275000/17946651057176172524/*/1or_QN1ksv81DNog6Tu_kWcZ5jJWf5W9E?e=download&uuid=93717736-fc70-4163-887e-3da69da89c26 [following]
--2022-07-27 11:11:24--  https://doc-0c-10-doc

In [3]:
# 결측치 처리

train.interpolate(inplace=True)
test.fillna(0, inplace=True)

In [4]:
# 보간법으로 결측치처리한 테스트셋에서 중요도가 낮은 피쳐 제거

x_train_1 = train.drop(['count', 'id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)
y_train = train['count']
test_1 = test.drop(['id', 'hour_bef_windspeed', 'hour_bef_pm2.5'], axis=1)

In [6]:
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor(criterion = 'mse', random_state=2020)

params = {'n_estimators': [200, 300, 500],
          'max_features': [5, 6, 8],
          'min_samples_leaf': [1, 3, 5]}

greedy_CV = GridSearchCV(model, param_grid=params, cv = 3, n_jobs = -1)
greedy_CV.fit(x_train_1,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
27 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/para

GridSearchCV(cv=3,
             estimator=RandomForestRegressor(criterion='mse',
                                             random_state=2020),
             n_jobs=-1,
             param_grid={'max_features': [5, 6, 8],
                         'min_samples_leaf': [1, 3, 5],
                         'n_estimators': [200, 300, 500]})

- 하이퍼파라미터 튜닝은 정지규칙 값들을 설정하는 것을 의미
- 의사결정나무에는 정지규칙(stopping criteria) 이라는 개념이 존재

- max_depth : 최대깊이는 최대로 내려갈 수 있는 depth 입니다. 뿌리 노드로부터 내려갈 수 있는 깊이를 지정하며 작을수록 트리는 작아지게 됩니다.

- min_samples_split : 최소 노드크기는 노드를 분할하기 위한 데이터 수 입니다. 해당 노드에 이 값보다 적은 확률변수 수가 있다면 stop. 작을수록 트리는 커지게 됩니다.

- min_impurity_decrease : 최소 향상도는 노드를 분할하기 위한 최소 향상도 입니다. 향상도가 설정값 이하라면 더 이상 분할하지 않습니다. 작을수록 트리는 커집니다.

- Cost-complexity : 비용 복잡도는 트리가 커지는 것에 대해 패널티 계수를 설정해서 불순도와 트리가 커지는 것에 대해 복잡도를 계산하는 것입니다.

- 하이퍼파라미터 튜닝 : 정지규칙들을 종합적으로 고려해 최적의 조건값을 설정하는 것

- 하이퍼파라미터 튜닝에는 다양한 방법론이 있다. 그 중 Best 성능을 나타내는 GridSearch는 완전 탐색(Exhaustive Search) 을 사용하는데, 가능한 모든 조합 중에서 가장 우수한 조합을 찾아주지만 완전 탐색이기 때문에 Best 조합을 찾을 때까지 시간이 매우 오래 걸린다는 단점이 있다.



In [7]:
pred = greedy_CV.predict(test_1)

In [8]:
pred

array([105.54333333, 214.12      ,  84.47      ,  40.77      ,
        59.20333333, 123.42333333, 194.43333333, 317.70666667,
        42.04      , 111.56333333, 295.17666667, 250.64      ,
        88.01666667,  41.04      , 208.23333333, 142.61      ,
        26.64      , 171.33333333, 344.54      , 153.84      ,
       232.16      ,  85.77333333,  25.5       , 140.47333333,
       142.61      , 114.72      ,  29.69666667, 116.35666667,
       112.89666667, 148.77      ,  75.32      ,  34.78333333,
        71.21      , 130.01      , 267.44333333,  40.01666667,
       129.84333333, 103.61666667, 244.90333333,  91.13      ,
        58.15      , 122.29      , 152.97      ,  91.05333333,
       333.61333333, 216.60333333,  94.61333333,  66.12666667,
        18.46666667,  85.43      , 226.15666667,  91.92666667,
       178.04      ,  82.18333333, 187.81666667, 131.50333333,
        51.48      , 171.97666667,  22.02      ,  20.24333333,
        88.26      ,  88.27      , 255.10333333, 307.49

In [9]:
submission_1 = pd.read_csv('data/submission.csv')

In [10]:
import numpy as np

submission_1['count'] = np.round(pred, 2)

In [12]:
submission_1.head()

,id,count
0,0,105.54
1,1,214.12
2,2,84.47
3,4,40.77
4,5,59.20


## 동일한 방법으로 처리한 테스트셋에서 중요도가 0.1 미만인 피쳐를 제거한 후 모델 사용

In [5]:
x_train_2= train.drop(['count','id', 'hour_bef_precipitation','hour_bef_windspeed','hour_bef_humidity','hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'], axis = 1)
test_2 = test.drop(['id', 'hour_bef_precipitation','hour_bef_windspeed','hour_bef_humidity','hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5'], axis = 1)

In [15]:
x_train_2

,hour,hour_bef_temperature,hour_bef_visibility
0,20,16.3,576.0
1,13,20.1,916.0
2,6,13.9,1382.0
3,23,8.1,946.0
4,18,29.5,2000.0
...,...,...,...
1454,4,16.8,2000.0
1455,3,10.8,2000.0
1456,5,18.3,2000.0
1457,21,20.7,1395.0


In [16]:
y_train

0        49.0
1       159.0
2        26.0
3        57.0
4       431.0
        ...  
1454     21.0
1455     20.0
1456     22.0
1457    216.0
1458    170.0
Name: count, Length: 1459, dtype: float64

In [17]:
model_2 = RandomForestRegressor(criterion = 'mse', random_state=2020)

params = {'n_estimators': [200, 300, 500],
          'max_features': [1,2,3],
          'min_samples_leaf': [1, 3, 5]}

greedy_CV = GridSearchCV(model_2, param_grid=params, cv = 3, n_jobs = -1)
greedy_CV.fit(x_train_2,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,


GridSearchCV(cv=3,
             estimator=RandomForestRegressor(criterion='mse',
                                             random_state=2020),
             n_jobs=-1,
             param_grid={'max_features': [1, 2, 3],
                         'min_samples_leaf': [1, 3, 5],
                         'n_estimators': [200, 300, 500]})

In [18]:
pred = greedy_CV.predict(test_2)

In [19]:
pred

array([103.39281506, 248.52843242, 101.85340717,  54.89043882,
        58.69715285, 125.29649228, 255.06493968, 309.2879934 ,
        49.60928207, 129.79003223, 289.28957351, 250.78575944,
       116.80589655,  51.79679016, 221.2629599 , 150.14026335,
        28.35012506, 231.34467242, 351.86668971, 174.51910544,
       199.23388408, 101.46966619,  43.83168224, 152.47087314,
       148.1020101 , 108.36834646,  28.48539731, 119.12254702,
       114.22619108, 160.97041546,  89.55020134,  46.26368591,
        72.25016919, 134.97380038, 259.54450622,  49.02430323,
       185.95006358, 214.60911977, 188.47979401,  88.78847992,
        57.64746578, 118.25405471, 147.21576587,  83.97217779,
       323.90599513, 207.23059339,  90.18376359,  64.75606703,
        17.26592532,  89.46509817, 233.75419312,  94.83802271,
       152.42392665, 143.56525992, 185.48879964, 147.42035041,
        57.9270475 , 176.19142507,  42.92872986,  22.92297403,
        93.41386616,  89.49096254, 254.6684145 , 301.61

In [20]:
submission_2 = pd.read_csv('data/submission.csv')

In [22]:
submission_2['count'] = np.round(pred,2)

In [23]:
submission_2.head()

,id,count
0,0,103.39
1,1,248.53
2,2,101.85
3,4,54.89
4,5,58.70
